In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import talib as ta

In [2]:
key_path = key_path
project = project_id

# Sources
table = 'bronze_ticker_data'
dataset = 'bronze'

# Tables id
table_conca = f'{project}.{dataset}.{table}'

# Loading tables
table_to_save = 'silver_indicators'
dataset_to_save = 'silver'
table_conca_to_save = f'{project}.{dataset_to_save}.{table_to_save}'


In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [7]:
df_all = bigquery.run_query(
    f"""
    SELECT
        *
    FROM {project}.{dataset}.{table}
    """
)
df_all

,date,open,high,low,close,adjclose,volume,ticker,id
0,2024-08-06 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,<NA>,EVBG,58a258ede842a18955064a445f21b74a
1,2024-07-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,<NA>,NWLI,2a108b0502eb44737e40d342735fb1cd
2,2022-09-13 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,<NA>,WHLRL,d47f0bb8ba0e78c6a58e3a4d10063ca3
3,2024-08-12 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,<NA>,HRT,6585569eb1810ccfad19366dee9df278
4,2022-12-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,<NA>,WHLRL,55b729b5388bfddb57d9fa66d0ac5d32
...,...,...,...,...,...,...,...,...,...
9142312,2024-03-25 00:00:00+00:00,35.950001,36.060001,35.070000,35.340000,35.340000,4316500,MTCH,f98d685056735faf09ee81e34370db96
9142313,2024-06-05 00:00:00+00:00,31.230000,31.670000,31.070000,31.440001,31.440001,4017500,MTCH,927a585f24168ef2a4966b88d2dd2468
9142314,2024-06-20 00:00:00+00:00,30.389999,30.544001,29.990000,30.290001,30.290001,3012700,MTCH,ed3a7e6d5f0107cf6465ff4f7703092b
9142315,2024-07-29 00:00:00+00:00,33.540001,34.049999,33.029999,33.200001,33.200001,6290000,MTCH,ba3388d7cac748ed4e052d877787851e


In [8]:
df_drop_nan = df_all.dropna()
df_drop_nan

,date,open,high,low,close,adjclose,volume,ticker,id
6,2023-08-07 00:00:00+00:00,1.560000,2.330000,1.560000,2.000000,2.000000,221800,BRFH,c4aadce35cab89ab104bd47d3b2bfce0
7,2023-05-25 00:00:00+00:00,1.900000,2.120000,1.900000,2.000000,2.000000,11700,YJ,3ca9d595b8dc27bc4c257a710c9be46e
8,2021-08-12 00:00:00+00:00,2.300000,2.400000,1.920000,2.000000,2.000000,266600,DWSN,925c9e404d659e43020cd22636dec824
9,2023-01-05 00:00:00+00:00,1.910000,2.000000,1.900000,2.000000,2.000000,20800,ARTW,1c3a81b49cc98314b5b703f87fe35ce1
10,2023-10-17 00:00:00+00:00,2.070000,2.090000,2.000000,2.000000,2.000000,11900,ERNA,23f1d08b19bd2eb7538c5c817fd47577
...,...,...,...,...,...,...,...,...,...
9142312,2024-03-25 00:00:00+00:00,35.950001,36.060001,35.070000,35.340000,35.340000,4316500,MTCH,f98d685056735faf09ee81e34370db96
9142313,2024-06-05 00:00:00+00:00,31.230000,31.670000,31.070000,31.440001,31.440001,4017500,MTCH,927a585f24168ef2a4966b88d2dd2468
9142314,2024-06-20 00:00:00+00:00,30.389999,30.544001,29.990000,30.290001,30.290001,3012700,MTCH,ed3a7e6d5f0107cf6465ff4f7703092b
9142315,2024-07-29 00:00:00+00:00,33.540001,34.049999,33.029999,33.200001,33.200001,6290000,MTCH,ba3388d7cac748ed4e052d877787851e


In [30]:
# Obtener el año actual
año_actual = datetime.now().year

# Agrupar por el campo 'ticker' y filtrar aquellos cuyo valor máximo de 'date' sea igual al año actual
tickers_filtrados = df_drop_nan.groupby('ticker').filter(lambda x: x['date'].max().year == año_actual)

# Filtramos aquellos tickers que tengan más de 5 años
tickers_filtrados = tickers_filtrados.groupby('ticker').count()
tickers_filtrados = tickers_filtrados[tickers_filtrados['date'] >= 252 * 5]

tickers_filtrados = tickers_filtrados.reset_index()
tickers_filtrados = tickers_filtrados['ticker']

tickers_filtrados


0          A
1       AACG
2       AADI
3       AADR
4        AAL
        ... 
3440     ZUO
3441    ZVRA
3442     ZWS
3443    ZYME
3444    ZYXI
Name: ticker, Length: 3445, dtype: object

In [34]:
df = pd.merge(df_drop_nan, tickers_filtrados, how='inner', on='ticker')
df = df[df.columns]
df = df.drop(columns=['year'])
df

,date,open,high,low,close,adjclose,volume,ticker,id
0,2023-08-07 00:00:00+00:00,1.560000,2.330000,1.560000,2.000000,2.000000,221800,BRFH,c4aadce35cab89ab104bd47d3b2bfce0
1,2024-07-05 00:00:00+00:00,3.300000,3.640000,3.300000,3.420000,3.420000,12000,BRFH,e16a0f4f764fe155a20f9d140a25100d
2,2023-05-15 00:00:00+00:00,1.400000,1.400000,1.220000,1.250000,1.250000,14400,BRFH,f9b1ee6a4f81197965675118bb430481
3,2022-07-27 00:00:00+00:00,4.250000,4.500000,4.250000,4.500000,4.500000,500,BRFH,672b34d116e337e0fb2d766196741eae
4,2015-03-13 00:00:00+00:00,6.240000,6.500000,6.240000,6.500000,6.500000,3569,BRFH,f0609067a39d8fb00202ac1ab61fbd48
...,...,...,...,...,...,...,...,...,...
7821003,2023-12-28 00:00:00+00:00,52.730000,53.750000,52.650002,53.669998,53.669998,2129300,TSN,ff4b946b6a21258a242f137b4e1d482f
7821004,2024-01-31 00:00:00+00:00,55.130001,55.320000,54.470001,54.759998,54.759998,4236300,TSN,8553cfacc22c81d22397a77f32085eee
7821005,2024-02-12 00:00:00+00:00,52.660000,52.990002,52.169998,52.549999,52.549999,3101200,TSN,8da7365a13f9c74fe9e3ff2b718e6b10
7821006,2024-05-01 00:00:00+00:00,60.360001,61.020000,59.980000,60.290001,60.290001,2370400,TSN,6e160a1756ee01517b9d803a202749f2


In [35]:
def calculate_indicators(df):
    # Indicadores técnicos
    df['SMA_20'] = ta.SMA(df['close'], timeperiod=20)
    df['EMA_50'] = ta.EMA(df['close'], timeperiod=50)
    df['WILLR'] = ta.WILLR(df['high'], df['low'], df['close'], timeperiod=14)
    df['ATR'] = ta.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['CCI'] = ta.CCI(df['high'], df['low'], df['close'], timeperiod=14)
    df['OBV'] = ta.OBV(df['close'], df['volume'])
    df['RSI'] = ta.RSI(df['close'], timeperiod=14)

    # Patrones de velas
    df['CDL_DOJI'] = ta.CDLDOJI(df['open'], df['high'], df['low'], df['close'])
    df['CDL_HAMMER'] = ta.CDLHAMMER(
        df['open'], df['high'], df['low'], df['close'])
    df['CDL_MORNING_STAR'] = ta.CDLMORNINGSTAR(
        df['open'], df['high'], df['low'], df['close'])
    df['CDL_SHOOTING_STAR'] = ta.CDLSHOOTINGSTAR(
        df['open'], df['high'], df['low'], df['close'])
    df['CDL_ENGULFING'] = ta.CDLENGULFING(
        df['open'], df['high'], df['low'], df['close'])
    df['CDL_LONGLINE'] = ta.CDLLONGLINE(
        df['open'], df['high'], df['low'], df['close'])

    # Otros indicadores adicionales para medir la volatilidad
    df['bb_bbh'], df['bb_bbm'], df['bb_bbl'] = ta.BBANDS(df['close'], timeperiod=20,
                   nbdevup=2, nbdevdn=2, matype=0)
    df['daily_return'] = df['close'].pct_change()
    df['macd'], df['macd_signal'], df['macd_hist'] = ta.MACD(df['close'], slowperiod=26,
                   fastperiod=12, signalperiod=9)
    df['stoch'], df['stoch_d'] = ta.STOCH(
        high=df['high'], low=df['low'], close=df['close'], fastk_period=21, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    
    return df

In [36]:
df = calculate_indicators(df)
df

,date,open,high,low,close,adjclose,volume,ticker,id,SMA_20,...,CDL_LONGLINE,bb_bbh,bb_bbm,bb_bbl,daily_return,macd,macd_signal,macd_hist,stoch,stoch_d
0,2023-08-07 00:00:00+00:00,1.560000,2.330000,1.560000,2.000000,2.000000,221800,BRFH,c4aadce35cab89ab104bd47d3b2bfce0,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-07-05 00:00:00+00:00,3.300000,3.640000,3.300000,3.420000,3.420000,12000,BRFH,e16a0f4f764fe155a20f9d140a25100d,NaN,...,0,NaN,NaN,NaN,0.710000,NaN,NaN,NaN,NaN,NaN
2,2023-05-15 00:00:00+00:00,1.400000,1.400000,1.220000,1.250000,1.250000,14400,BRFH,f9b1ee6a4f81197965675118bb430481,NaN,...,0,NaN,NaN,NaN,-0.634503,NaN,NaN,NaN,NaN,NaN
3,2022-07-27 00:00:00+00:00,4.250000,4.500000,4.250000,4.500000,4.500000,500,BRFH,672b34d116e337e0fb2d766196741eae,NaN,...,0,NaN,NaN,NaN,2.600000,NaN,NaN,NaN,NaN,NaN
4,2015-03-13 00:00:00+00:00,6.240000,6.500000,6.240000,6.500000,6.500000,3569,BRFH,f0609067a39d8fb00202ac1ab61fbd48,NaN,...,0,NaN,NaN,NaN,0.444444,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7821003,2023-12-28 00:00:00+00:00,52.730000,53.750000,52.650002,53.669998,53.669998,2129300,TSN,ff4b946b6a21258a242f137b4e1d482f,66.497000,...,100,580.913349,66.497000,-447.919350,0.035900,-7.764849,-6.191727,-1.573122,14.923729,9.941363
7821004,2024-01-31 00:00:00+00:00,55.130001,55.320000,54.470001,54.759998,54.759998,4236300,TSN,8553cfacc22c81d22397a77f32085eee,64.852999,...,0,579.198716,64.852999,-449.492717,0.020309,-7.343572,-6.422096,-0.921476,16.501137,13.610302
7821005,2024-02-12 00:00:00+00:00,52.660000,52.990002,52.169998,52.549999,52.549999,3101200,TSN,8da7365a13f9c74fe9e3ff2b718e6b10,62.726499,...,0,576.906396,62.726499,-451.453398,-0.040358,-7.106120,-6.558901,-0.547219,16.981499,16.135455
7821006,2024-05-01 00:00:00+00:00,60.360001,61.020000,59.980000,60.290001,60.290001,2370400,TSN,6e160a1756ee01517b9d803a202749f2,61.375500,...,0,575.431900,61.375500,-452.680901,0.147288,-6.221665,-6.491454,0.269788,23.114939,18.865858


In [40]:
nulos_por_columna = df.isnull().sum()
nulos_por_columna

date                  0
open                  0
high                  0
low                   0
close                 0
adjclose              0
volume                0
ticker                0
id                    0
SMA_20               19
EMA_50               49
WILLR                13
ATR                  14
ADX                  27
CCI                  13
OBV                   0
RSI                  14
CDL_DOJI              0
CDL_HAMMER            0
CDL_MORNING_STAR      0
CDL_SHOOTING_STAR     0
CDL_ENGULFING         0
CDL_LONGLINE          0
bb_bbh               19
bb_bbm               19
bb_bbl               19
daily_return          1
macd                 33
macd_signal          33
macd_hist            33
stoch                24
stoch_d              24
dtype: int64

In [42]:
df = df.fillna(0)
nulos_por_columna = df.isnull().sum()
nulos_por_columna

date                 0
open                 0
high                 0
low                  0
close                0
adjclose             0
volume               0
ticker               0
id                   0
SMA_20               0
EMA_50               0
WILLR                0
ATR                  0
ADX                  0
CCI                  0
OBV                  0
RSI                  0
CDL_DOJI             0
CDL_HAMMER           0
CDL_MORNING_STAR     0
CDL_SHOOTING_STAR    0
CDL_ENGULFING        0
CDL_LONGLINE         0
bb_bbh               0
bb_bbm               0
bb_bbl               0
daily_return         0
macd                 0
macd_signal          0
macd_hist            0
stoch                0
stoch_d              0
dtype: int64

In [45]:
# En el caso de tener un df en Bigquery, lo leemos y guardamos solo los nuevos registros
try:
    # Filtramos solamente los nuevos registros
    df_incremental = bigquery.select_for_incremental(id='id', table=table_conca_to_save, new_df=df)

    # Guardamos los datos en bigquery
    bigquery.save_dataframe(df_incremental, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)

# En el caso de no tener datos en Bigquery, guardamos todo el df
except:
    bigquery.save_dataframe(df, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
